In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def scrape_chart(url, week):
    src = requests.get(url).text
    soup = BeautifulSoup(src, 'lxml')
    rows = soup.select('.o-chart-results-list-row') # Pull out the ul for each chart row
    songs = {'rank': [], 'title': [], 'artist': []}

    for row in rows:
        songs['rank'].append(row.li.find('span').get_text().strip())
        songs['title'].append(row.find('h3').get_text().strip())
        songs['artist'].append(row.find_all('li')[3].find('span').get_text().strip())    
    
    songs['week'] = 100 * [week]
    return pd.DataFrame(songs)


In [97]:
ts = pd.Timestamp('2022-03-08')
chart = pd.DataFrame(columns=['rank', 'title', 'artist', 'week'])

while ts < pd.Timestamp.now():
    week = ts.strftime('%Y-%m-%d')
    print(week)
    url = f'https://www.billboard.com/charts/hot-100/{week}'
    toadd = scrape_chart(url, week)
    print(toadd.head())
    pd.concat([chart, toadd])
    print(chart.head())
    ts += pd.Timedelta(days=7)


2022-03-08
  rank                      title  \
0    1                 Heat Waves   
1    2  We Don't Talk About Bruno   
2    3                    abcdefu   
3    4              Super Gremlin   
4    5                       Stay   

                                              artist  
0                                      Glass Animals  
1  Carolina Gaitan, Mauro Castillo, Adassa, Rhenz...  
2                                              GAYLE  
3                                        Kodak Black  
4                      The Kid LAROI & Justin Bieber  
Empty DataFrame
Columns: [rank, title, artist, week]
Index: []
2022-03-15
  rank                      title  \
0    1                 Heat Waves   
1    2  We Don't Talk About Bruno   
2    3              Super Gremlin   
3    4                    abcdefu   
4    5                       Stay   

                                              artist  
0                                      Glass Animals  
1  Carolina Gaitan, Mauro Cas

In [3]:
loaded = pd.read_feather('charts.feather')
loaded.head()

,index,rank,title,artist,week
0,0,1,Poor Little Fool,Ricky Nelson,1958-08-04
1,1,2,Patricia,Perez Prado And His Orchestra,1958-08-04
2,2,3,Splish Splash,Bobby Darin,1958-08-04
3,3,4,Hard Headed Woman,Elvis Presley With The Jordanaires,1958-08-04
4,4,5,When,Kalin Twins,1958-08-04
